# scRFE Tutorial


Here we present an example of how to use scRFE. We analyze the Limb Muscle Facs data from the Tabula-Muris-Senis dataset that is available on Figshare. We split the data by age.

More features were selected than ideal in this model, because we used a very small number of estimators and a low CV score, for time's sake. This results are not accurate though, and we recommend running the code with 1000 estimators and CV>=5 with an EC2 instance.

### Imports 

In [41]:
# Imports 
import numpy as np
import pandas as pd
import scanpy as sc
from anndata import read_h5ad
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import SelectFromModel
from sklearn.metrics import accuracy_score
from sklearn.feature_selection import RFE
from sklearn.feature_selection import RFECV

### Read in anndata file 

In [58]:
cd

/Users/madelinepark


In [101]:
# read in data 
# adata = read_h5ad('/Users/madelinepark/Downloads/Kidney_facs.h5ad')
adata = read_h5ad('/Users/madelinepark/Downloads/Mammary_Gland_droplet.h5ad')
tiss = adata
# tiss.obs

In [99]:
cd /Users/madelinepark/Desktop

/Users/madelinepark/Desktop


In [100]:
for c in list(set(tiss.obs['age'])): 
    print(c)
    age_of_interest = c
    tiss.obs['age_type_of_interest'] = 'rest' #OMG SO IT WAS IN THE WRONG SPOT!!!

    tiss.obs.loc[tiss.obs['age'] == age_of_interest,'age_type_of_interest'] = age_of_interest
tiss.obs

3m
21m
18m


,age,batch,cell,cell_ontology_class,cell_ontology_id,free_annotation,method,mouse.id,n_genes,sex,subtissue,tissue,tissue_free_annotation,n_counts,louvain,cluster_names,age_type_of_interest
index,,,,,,,,,,,,,,,,,
AAACCTGGTGCTGTAT-1-4-0-0,18m,0,MACA_18m_F_BREAST_50_AAACCTGGTGCTGTAT,luminal epithelial cell of mammary gland,CL:0002326,luminal progenitor cell,droplet,18-F-50,2448,female,NA,Mammary_Gland,Mammary_Gland,8979.0,4,4_luminal epithelial cell of mammary gland,18m
AAACGGGAGGGTATCG-1-4-0-0,18m,0,MACA_18m_F_BREAST_50_AAACGGGAGGGTATCG,luminal epithelial cell of mammary gland,CL:0002326,luminal progenitor cell,droplet,18-F-50,2395,female,NA,Mammary_Gland,Mammary_Gland,8939.0,4,4_luminal epithelial cell of mammary gland,18m
AAACGGGGTAAGTTCC-1-4-0-0,18m,0,MACA_18m_F_BREAST_50_AAACGGGGTAAGTTCC,B cell,CL:0000236,B cell,droplet,18-F-50,1966,female,NA,Mammary_Gland,Mammary_Gland,7900.0,1,1_B cell,18m
AAACGGGTCGAATGGG-1-4-0-0,18m,0,MACA_18m_F_BREAST_50_AAACGGGTCGAATGGG,B cell,CL:0000236,B cell,droplet,18-F-50,1252,female,NA,Mammary_Gland,Mammary_Gland,3197.0,19,19_B cell,18m
AAAGATGGTAGTGAAT-1-4-0-0,18m,0,MACA_18m_F_BREAST_50_AAAGATGGTAGTGAAT,stromal cell,CL:0000499,stromal cell,droplet,18-F-50,1825,female,NA,Mammary_Gland,Mammary_Gland,5663.0,6,6_stromal cell,18m
AAAGATGGTTGTCTTT-1-4-0-0,18m,0,MACA_18m_F_BREAST_50_AAAGATGGTTGTCTTT,stromal cell,CL:0000499,stromal cell,droplet,18-F-50,1829,female,NA,Mammary_Gland,Mammary_Gland,5429.0,6,6_stromal cell,18m
AAAGATGTCGGATGTT-1-4-0-0,18m,0,MACA_18m_F_BREAST_50_AAAGATGTCGGATGTT,luminal epithelial cell of mammary gland,CL:0002326,luminal epithelial cell of mammary gland,droplet,18-F-50,2507,female,NA,Mammary_Gland,Mammary_Gland,8635.0,5,5_luminal epithelial cell of mammary gland,18m
AAAGCAAAGATCTGCT-1-4-0-0,18m,0,MACA_18m_F_BREAST_50_AAAGCAAAGATCTGCT,luminal epithelial cell of mammary gland,CL:0002326,luminal epithelial cell of mammary gland,droplet,18-F-50,2532,female,NA,Mammary_Gland,Mammary_Gland,8683.0,5,5_luminal epithelial cell of mammary gland,18m
AAAGTAGAGGCCGAAT-1-4-0-0,18m,0,MACA_18m_F_BREAST_50_AAAGTAGAGGCCGAAT,luminal epithelial cell of mammary gland,CL:0002326,luminal progenitor cell,droplet,18-F-50,1718,female,NA,Mammary_Gland,Mammary_Gland,6032.0,4,4_luminal epithelial cell of mammary gland,18m


### Run scRFE one vs all classification for age

In [102]:
results_age_cv = pd.DataFrame() #create results data frame 

for c in list(set(tiss.obs['age'])): 
    print(c)
    age_of_interest = c
    tiss.obs['age_type_of_interest'] = 'rest' #moved into, was outside before
    tiss.obs.loc[tiss.obs['age'] == age_of_interest,'age_type_of_interest'] = age_of_interest

    clf = RandomForestClassifier(n_estimators=10, random_state=0, n_jobs=-1, oob_score=True)
    selector = RFECV(clf, step=0.2, cv=3, n_jobs=4) # step = % rounded down at each iteration  
    
# #     tiss.obs.loc[tiss.obs[tiss.obs['age'] == age_of_interest].index,'age_type_of_interest'] = age_of_interest
#     tiss.obs.loc[tiss.obs['age'] == age_of_interest,'age_type_of_interest'] = age_of_interest

    feat_labels = tiss.var_names 
    X = tiss.X
    y = tiss.obs['age_type_of_interest']
    
    print('training...')
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.05, random_state=0) 
    clf.fit(X_train, y_train)
    selector.fit(X_train, y_train)
    feature_selected = feat_labels[selector.support_] 
    
    print('result writing')
    column_headings = []
    column_headings.append(c)
    column_headings.append(c + '_gini')
    
    resaux = pd.DataFrame(columns=column_headings)
    resaux[c] = feature_selected
    resaux[c + '_gini'] = (selector.estimator_.feature_importances_)
    
    print(feature_selected)
    print (selector.estimator_.feature_importances_)
    
    results_age_cv = pd.concat([results_age_cv,resaux],axis=1)
    
    tiss.obs['age_type_of_interest'] = 'rest'
    
results_age_cv

3m
training...


/Users/madelinepark/anaconda3/envs/scanpytest/lib/python3.6/site-packages/sklearn/ensemble/forest.py:460: UserWarning: Some inputs do not have OOB scores. This probably means too few trees were used to compute any reliable oob estimates.
  warn("Some inputs do not have OOB scores. "
/Users/madelinepark/anaconda3/envs/scanpytest/lib/python3.6/site-packages/sklearn/ensemble/forest.py:465: RuntimeWarning: invalid value encountered in true_divide
  predictions[k].sum(axis=1)[:, np.newaxis])
/Users/madelinepark/anaconda3/envs/scanpytest/lib/python3.6/site-packages/sklearn/ensemble/forest.py:460: UserWarning: Some inputs do not have OOB scores. This probably means too few trees were used to compute any reliable oob estimates.
  warn("Some inputs do not have OOB scores. "
/Users/madelinepark/anaconda3/envs/scanpytest/lib/python3.6/site-packages/sklearn/ensemble/forest.py:465: RuntimeWarning: invalid value encountered in true_divide
  predictions[k].sum(axis=1)[:, np.newaxis])
/Users/madelinep

result writing
Index(['Sox17', 'Mrpl15', 'Lypla1', 'Tcea1', 'Atp6v1h', 'Rb1cc1', 'Pcmtd1',
       'Sntg1', 'Rrs1', 'Adhfe1',
       ...
       'Figf', 'Ofd1', 'Rab9', 'Tceanc', 'Tmsb4x', 'Msl3', 'Mid1',
       'G530011O06Rik', 'Vamp7', 'Erdr1'],
      dtype='object', name='index', length=3972)
[1.60192509e-04 1.79927973e-04 2.64630913e-04 ... 0.00000000e+00
 9.43876748e-05 3.69572201e-05]
21m
training...


/Users/madelinepark/anaconda3/envs/scanpytest/lib/python3.6/site-packages/sklearn/ensemble/forest.py:460: UserWarning: Some inputs do not have OOB scores. This probably means too few trees were used to compute any reliable oob estimates.
  warn("Some inputs do not have OOB scores. "
/Users/madelinepark/anaconda3/envs/scanpytest/lib/python3.6/site-packages/sklearn/ensemble/forest.py:465: RuntimeWarning: invalid value encountered in true_divide
  predictions[k].sum(axis=1)[:, np.newaxis])
/Users/madelinepark/anaconda3/envs/scanpytest/lib/python3.6/site-packages/sklearn/ensemble/forest.py:460: UserWarning: Some inputs do not have OOB scores. This probably means too few trees were used to compute any reliable oob estimates.
  warn("Some inputs do not have OOB scores. "
/Users/madelinepark/anaconda3/envs/scanpytest/lib/python3.6/site-packages/sklearn/ensemble/forest.py:465: RuntimeWarning: invalid value encountered in true_divide
  predictions[k].sum(axis=1)[:, np.newaxis])
/Users/madelinep

result writing
Index(['Mrpl15', 'Lypla1', 'Tcea1', 'Atp6v1h', 'Rb1cc1', 'Pcmtd1', 'Rrs1',
       'Mybl1', 'Snhg6', 'Cops5',
       ...
       'Ctps2', 'Ofd1', 'Rab9', 'Tmsb4x', 'Prps2', 'Msl3', 'Hccs', 'Mid1',
       'G530011O06Rik', 'Erdr1'],
      dtype='object', name='index', length=3972)
[9.29968863e-05 0.00000000e+00 0.00000000e+00 ... 7.15805230e-05
 3.91390250e-04 1.59400719e-04]
18m
training...


/Users/madelinepark/anaconda3/envs/scanpytest/lib/python3.6/site-packages/sklearn/ensemble/forest.py:460: UserWarning: Some inputs do not have OOB scores. This probably means too few trees were used to compute any reliable oob estimates.
  warn("Some inputs do not have OOB scores. "
/Users/madelinepark/anaconda3/envs/scanpytest/lib/python3.6/site-packages/sklearn/ensemble/forest.py:465: RuntimeWarning: invalid value encountered in true_divide
  predictions[k].sum(axis=1)[:, np.newaxis])
/Users/madelinepark/anaconda3/envs/scanpytest/lib/python3.6/site-packages/sklearn/ensemble/forest.py:460: UserWarning: Some inputs do not have OOB scores. This probably means too few trees were used to compute any reliable oob estimates.
  warn("Some inputs do not have OOB scores. "
/Users/madelinepark/anaconda3/envs/scanpytest/lib/python3.6/site-packages/sklearn/ensemble/forest.py:465: RuntimeWarning: invalid value encountered in true_divide
  predictions[k].sum(axis=1)[:, np.newaxis])
/Users/madelinep

result writing
Index(['Mrpl15', 'Lypla1', 'Tcea1', 'Rgs20', 'Atp6v1h', 'Rb1cc1', 'Rrs1',
       'Cops5', 'Cspp1', 'Cpa6',
       ...
       'Morf4l2', 'Tbc1d8b', 'Huwe1', 'Sat1', 'Prdx4', 'Eif1ax', 'Pdha1',
       'Tmsb4x', 'Hccs', 'G530011O06Rik'],
      dtype='object', name='index', length=3972)
[3.53685773e-05 1.11584380e-04 0.00000000e+00 ... 8.80283298e-04
 9.86796168e-05 4.75630737e-05]


,3m,3m_gini,21m,21m_gini,18m,18m_gini
0,Sox17,0.000160,Mrpl15,0.000093,Mrpl15,0.000035
1,Mrpl15,0.000180,Lypla1,0.000000,Lypla1,0.000112
2,Lypla1,0.000265,Tcea1,0.000000,Tcea1,0.000000
3,Tcea1,0.000484,Atp6v1h,0.000163,Rgs20,0.000000
4,Atp6v1h,0.000139,Rb1cc1,0.000000,Atp6v1h,0.000153
5,Rb1cc1,0.000000,Pcmtd1,0.000089,Rb1cc1,0.000000
6,Pcmtd1,0.000138,Rrs1,0.000330,Rrs1,0.000000
7,Sntg1,0.000000,Mybl1,0.000000,Cops5,0.000052
8,Rrs1,0.000829,Snhg6,0.000257,Cspp1,0.000103
9,Adhfe1,0.000000,Cops5,0.000089,Cpa6,0.000000


In [87]:
results_age_cv = pd.DataFrame() #create results data frame 


for c in list(set(tiss.obs['age'])): 
    print(c)
    age_of_interest = c
    tiss.obs['age_type_of_interest'] = 'rest' #moved into, was outside before
    tiss.obs.loc[tiss.obs['age'] == age_of_interest,'age_type_of_interest'] = age_of_interest

    clf = RandomForestClassifier(n_estimators=1000, random_state=0, n_jobs=-1, oob_score=True)
    selector = RFECV(clf, step=0.2, cv=3, n_jobs=4) # step = % rounded down at each iteration  
    
# #     tiss.obs.loc[tiss.obs[tiss.obs['age'] == age_of_interest].index,'age_type_of_interest'] = age_of_interest
#     tiss.obs.loc[tiss.obs['age'] == age_of_interest,'age_type_of_interest'] = age_of_interest

    feat_labels = tiss.var_names 
    X = tiss.X
    y = tiss.obs['age_type_of_interest']
    
    print('training...')
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.05, random_state=0) 
    clf.fit(X_train, y_train)
    selector.fit(X_train, y_train)
    feature_selected = feat_labels[selector.support_] 
    
    print('result writing')
    column_headings = []
    column_headings.append(c)
    column_headings.append(c + '_gini')
    
    resaux = pd.DataFrame(columns=column_headings)
    resaux[c] = feature_selected
    resaux[c + '_gini'] = (selector.estimator_.feature_importances_)
    
    print(feature_selected)
    print (selector.estimator_.feature_importances_)
    
    results_age_cv = pd.concat([results_age_cv,resaux],axis=1)
    
#     tiss.obs['age_type_of_interest'] = 'rest'
    
results_age_cv

24m
training...
result writing
Index(['0610005C13Rik', '0610007C21Rik', '0610007N19Rik', '0610009B22Rik',
       '0610009D07Rik', '0610010K14Rik', '0610010O12Rik', '0610011F06Rik',
       '0610011L14Rik', '0610012G03Rik',
       ...
       'Znhit2-ps', 'Znhit6', 'Znrd1', 'Znrf1', 'Znrf2', 'Zranb2', 'Zrsr2',
       'Zwint', 'Zyg11b', 'Zyx'],
      dtype='object', name='index', length=4583)
[4.63119127e-05 1.36097809e-04 8.04043033e-05 ... 5.12499908e-05
 1.15301364e-04 6.56965168e-05]
3m
training...
result writing
Index(['0610005C13Rik', '0610007C21Rik', '0610007N19Rik', '0610009B22Rik',
       '0610009D07Rik', '0610010K14Rik', '0610010O12Rik', '0610011F06Rik',
       '0610011L14Rik', '0610012G03Rik',
       ...
       'Znhit2-ps', 'Znhit6', 'Znrd1', 'Znrf1', 'Znrf2', 'Zranb2', 'Zrsr2',
       'Zwint', 'Zyg11b', 'Zyx'],
      dtype='object', name='index', length=4583)
[4.63119127e-05 1.36097809e-04 8.04043033e-05 ... 5.12499908e-05
 1.15301364e-04 6.56965168e-05]


,24m,24m_gini,3m,3m_gini
0,0610005C13Rik,0.000046,0610005C13Rik,0.000046
1,0610007C21Rik,0.000136,0610007C21Rik,0.000136
2,0610007N19Rik,0.000080,0610007N19Rik,0.000080
3,0610009B22Rik,0.000066,0610009B22Rik,0.000066
4,0610009D07Rik,0.000060,0610009D07Rik,0.000060
5,0610010K14Rik,0.000035,0610010K14Rik,0.000035
6,0610010O12Rik,0.000110,0610010O12Rik,0.000110
7,0610011F06Rik,0.000143,0610011F06Rik,0.000143
8,0610011L14Rik,0.000030,0610011L14Rik,0.000030
9,0610012G03Rik,0.000073,0610012G03Rik,0.000073


In [85]:
results_age_cv.to_csv('test7.csv')

In [93]:
# read in data 
adata = read_h5ad('/Users/madelinepark/Downloads/Kidney_facs.h5ad')
tiss2 = adata
results_age_first = pd.DataFrame()

for c in list(set(tiss2.obs['age'])):
    print(c)    
    
    age_of_interest = c
    tiss2.obs['age_type_of_interest'] = 'rest'

    tiss2.obs.loc[tiss2.obs['age'] == age_of_interest,'age_type_of_interest'] = age_of_interest
    tiss2.obs['age_type_of_interest'] = 'rest'


24m
3m


In [94]:
tiss2.obs

,FACS.selection,age,batch,cell,cell_ontology_class,cell_ontology_id,cellid,free_annotation,method,mouse.id,plate,sex,subtissue,tissue,well,n_genes,n_counts,louvain,cluster_names,age_type_of_interest
index,,,,,,,,,,,,,,,,,,,,
A11_B001261_S287_L003.mus-2-0,Viable,24m,0,nan,T cell,CL:0000084,A11_B001261,T cell,facs,24_59_M,B001261,male,NA,Kidney,A11,1488,1108365.0,8,8_T cell,rest
A12_B001261_S288_L003.mus-2-0,Viable,24m,0,nan,epithelial cell of proximal tubule,CL:0002306,A12_B001261,Proximal tube,facs,24_59_M,B001261,male,NA,Kidney,A12,1855,2755523.0,5,5_epithelial cell of proximal tubule,rest
A15_B001261_S291_L003.mus-2-0,Viable,24m,0,nan,kidney collecting duct principal cell,CL:1001431,A15_B001261,ductal princeple,facs,24_59_M,B001261,male,NA,Kidney,A15,718,708214.0,7,7_kidney collecting duct principal cell,rest
A16_B001261_S292_L003.mus-2-0,Viable,24m,0,nan,T cell,CL:0000084,A16_B001261,T cell,facs,24_59_M,B001261,male,NA,Kidney,A16,2113,2041432.0,8,8_T cell,rest
A17_B001261_S293_L003.mus-2-0,Viable,24m,0,nan,epithelial cell of proximal tubule,CL:0002306,A17_B001261,Proximal tube,facs,24_59_M,B001261,male,NA,Kidney,A17,1926,1743637.0,5,5_epithelial cell of proximal tubule,rest
A18_B001261_S294_L003.mus-2-0,Viable,24m,0,nan,macrophage,CL:0000235,A18_B001261,macrophage,facs,24_59_M,B001261,male,NA,Kidney,A18,1722,1671482.0,6,6_macrophage,rest
A20_B001261_S296_L003.mus-2-0,Viable,24m,0,nan,epithelial cell of proximal tubule,CL:0002306,A20_B001261,Proximal tube,facs,24_59_M,B001261,male,NA,Kidney,A20,1327,1335461.0,5,5_epithelial cell of proximal tubule,rest
A21_B001261_S297_L003.mus-2-0,Viable,24m,0,nan,T cell,CL:0000084,A21_B001261,T cell,facs,24_59_M,B001261,male,NA,Kidney,A21,1991,1374989.0,8,8_T cell,rest
A3_B001261_S279_L003.mus-2-0,Viable,24m,0,nan,epithelial cell of proximal tubule,CL:0002306,A3_B001261,Proximal tube,facs,24_59_M,B001261,male,NA,Kidney,A3,1358,953170.0,2,2_epithelial cell of proximal tubule,rest


### Run scRFE one vs all classification for cell type

In [40]:
tiss.obs['cell_type_of_interest'] = 'rest'
results_cell_cv = pd.DataFrame() #create results data frame 

for c in list(set(tiss.obs['cell_ontology_class'])): 
    print(c)
    clf = RandomForestClassifier(n_estimators=10, random_state=0, n_jobs=-1, oob_score=True)
    selector = RFECV(clf, step=0.2, cv=3, n_jobs=4) # step = % rounded down at each iteration  
    age_of_interest = c
    
#     tiss.obs.loc[tiss.obs[tiss.obs['age'] == age_of_interest].index,'age_type_of_interest'] = age_of_interest
    tiss.obs.loc[tiss.obs['cell_ontology_class'] == cell_of_interest,'cell_type_of_interest'] = cell_of_interest 
    
tiss.obs

Kidney-new-24m-cluster
kidney loop of Henle ascending limb epithelial cell
fenestrated cell
mesangial cell
B cell
kidney collecting duct epithelial cell
T cell
kidney collecting duct principal cell
macrophage
epithelial cell of proximal tubule


,FACS.selection,age,batch,cell,cell_ontology_class,cell_ontology_id,cellid,free_annotation,method,mouse.id,plate,sex,subtissue,tissue,well,n_genes,n_counts,louvain,cluster_names,cell_type_of_interest
index,,,,,,,,,,,,,,,,,,,,
A11_B001261_S287_L003.mus-2-0,Viable,24m,0,nan,T cell,CL:0000084,A11_B001261,T cell,facs,24_59_M,B001261,male,NA,Kidney,A11,1488,1108365.0,8,8_T cell,rest
A12_B001261_S288_L003.mus-2-0,Viable,24m,0,nan,epithelial cell of proximal tubule,CL:0002306,A12_B001261,Proximal tube,facs,24_59_M,B001261,male,NA,Kidney,A12,1855,2755523.0,5,5_epithelial cell of proximal tubule,rest
A15_B001261_S291_L003.mus-2-0,Viable,24m,0,nan,kidney collecting duct principal cell,CL:1001431,A15_B001261,ductal princeple,facs,24_59_M,B001261,male,NA,Kidney,A15,718,708214.0,7,7_kidney collecting duct principal cell,rest
A16_B001261_S292_L003.mus-2-0,Viable,24m,0,nan,T cell,CL:0000084,A16_B001261,T cell,facs,24_59_M,B001261,male,NA,Kidney,A16,2113,2041432.0,8,8_T cell,rest
A17_B001261_S293_L003.mus-2-0,Viable,24m,0,nan,epithelial cell of proximal tubule,CL:0002306,A17_B001261,Proximal tube,facs,24_59_M,B001261,male,NA,Kidney,A17,1926,1743637.0,5,5_epithelial cell of proximal tubule,rest
A18_B001261_S294_L003.mus-2-0,Viable,24m,0,nan,macrophage,CL:0000235,A18_B001261,macrophage,facs,24_59_M,B001261,male,NA,Kidney,A18,1722,1671482.0,6,6_macrophage,rest
A20_B001261_S296_L003.mus-2-0,Viable,24m,0,nan,epithelial cell of proximal tubule,CL:0002306,A20_B001261,Proximal tube,facs,24_59_M,B001261,male,NA,Kidney,A20,1327,1335461.0,5,5_epithelial cell of proximal tubule,rest
A21_B001261_S297_L003.mus-2-0,Viable,24m,0,nan,T cell,CL:0000084,A21_B001261,T cell,facs,24_59_M,B001261,male,NA,Kidney,A21,1991,1374989.0,8,8_T cell,rest
A3_B001261_S279_L003.mus-2-0,Viable,24m,0,nan,epithelial cell of proximal tubule,CL:0002306,A3_B001261,Proximal tube,facs,24_59_M,B001261,male,NA,Kidney,A3,1358,953170.0,2,2_epithelial cell of proximal tubule,rest


In [34]:
results_cell_cv = pd.DataFrame() #create results data frame 

for c in list(set(tiss.obs['cell_ontology_class'])): 
    print(c)
    clf = RandomForestClassifier(n_estimators=10, random_state=0, n_jobs=-1, oob_score=True)
    selector = RFECV(clf, step=0.2, cv=3, n_jobs=4) # step = % rounded down at each iteration  
    age_of_interest = c
    tiss.obs['cell_type_of_interest'] = 'rest'

#     tiss.obs.loc[tiss.obs[tiss.obs['age'] == age_of_interest].index,'age_type_of_interest'] = age_of_interest
    tiss.obs.loc[tiss.obs['cell_ontology_class'] == cell_of_interest,'cell_type_of_interest'] = cell_of_interest

    
    feat_labels = tiss.var_names 
    X = tiss.X
    y = tiss.obs['cell_type_of_interest']
    
    print('training...')
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.05, random_state=0) 
    clf.fit(X_train, y_train)
    selector.fit(X_train, y_train)
    feature_selected = feat_labels[selector.support_] 
    
    print('result writing')
    column_headings = []
    column_headings.append(c)
    column_headings.append(c + '_gini')
    
    resaux = pd.DataFrame(columns=column_headings)
    resaux[c] = feature_selected
    resaux[c + '_gini'] = (selector.estimator_.feature_importances_)
    
    print(feature_selected)
    print (selector.estimator_.feature_importances_)
    
    results_cell_cv = pd.concat([results_cell_cv,resaux],axis=1)
    
    tiss.obs['cell_type_of_interest'] = 'rest'
    
results_cell_cv

Kidney-new-24m-cluster
training...


/Users/madelinepark/anaconda3/envs/scanpytest/lib/python3.6/site-packages/sklearn/ensemble/forest.py:460: UserWarning: Some inputs do not have OOB scores. This probably means too few trees were used to compute any reliable oob estimates.
  warn("Some inputs do not have OOB scores. "
/Users/madelinepark/anaconda3/envs/scanpytest/lib/python3.6/site-packages/sklearn/ensemble/forest.py:465: RuntimeWarning: invalid value encountered in true_divide
  predictions[k].sum(axis=1)[:, np.newaxis])
/Users/madelinepark/anaconda3/envs/scanpytest/lib/python3.6/site-packages/sklearn/ensemble/forest.py:460: UserWarning: Some inputs do not have OOB scores. This probably means too few trees were used to compute any reliable oob estimates.
  warn("Some inputs do not have OOB scores. "
/Users/madelinepark/anaconda3/envs/scanpytest/lib/python3.6/site-packages/sklearn/ensemble/forest.py:465: RuntimeWarning: invalid value encountered in true_divide
  predictions[k].sum(axis=1)[:, np.newaxis])
/Users/madelinep

result writing
Index(['0610007N19Rik', '0610007P08Rik', '0610007P14Rik', '0610007P22Rik',
       '0610008F07Rik', '0610009B14Rik', '0610009B22Rik', '0610009D07Rik',
       '0610009L18Rik', '0610009O20Rik',
       ...
       'Zrsr1', 'Zrsr2', 'Zscan10', 'Zscan12', 'Zscan18', 'Zscan2', 'Zscan20',
       'Zscan21', 'Zscan29', 'zsGreen_transgene'],
      dtype='object', name='index', length=9162)
[0. 0. 0. ... 0. 0. 0.]
kidney loop of Henle ascending limb epithelial cell
training...


/Users/madelinepark/anaconda3/envs/scanpytest/lib/python3.6/site-packages/sklearn/ensemble/forest.py:460: UserWarning: Some inputs do not have OOB scores. This probably means too few trees were used to compute any reliable oob estimates.
  warn("Some inputs do not have OOB scores. "
/Users/madelinepark/anaconda3/envs/scanpytest/lib/python3.6/site-packages/sklearn/ensemble/forest.py:465: RuntimeWarning: invalid value encountered in true_divide
  predictions[k].sum(axis=1)[:, np.newaxis])
/Users/madelinepark/anaconda3/envs/scanpytest/lib/python3.6/site-packages/sklearn/ensemble/forest.py:460: UserWarning: Some inputs do not have OOB scores. This probably means too few trees were used to compute any reliable oob estimates.
  warn("Some inputs do not have OOB scores. "
/Users/madelinepark/anaconda3/envs/scanpytest/lib/python3.6/site-packages/sklearn/ensemble/forest.py:465: RuntimeWarning: invalid value encountered in true_divide
  predictions[k].sum(axis=1)[:, np.newaxis])
/Users/madelinep

result writing
Index(['0610007N19Rik', '0610007P08Rik', '0610007P14Rik', '0610007P22Rik',
       '0610008F07Rik', '0610009B14Rik', '0610009B22Rik', '0610009D07Rik',
       '0610009L18Rik', '0610009O20Rik',
       ...
       'Zrsr1', 'Zrsr2', 'Zscan10', 'Zscan12', 'Zscan18', 'Zscan2', 'Zscan20',
       'Zscan21', 'Zscan29', 'zsGreen_transgene'],
      dtype='object', name='index', length=9162)
[0. 0. 0. ... 0. 0. 0.]
fenestrated cell
training...


...

/Users/madelinepark/anaconda3/envs/scanpytest/lib/python3.6/site-packages/sklearn/ensemble/forest.py:460: UserWarning: Some inputs do not have OOB scores. This probably means too few trees were used to compute any reliable oob estimates.
  warn("Some inputs do not have OOB scores. "
/Users/madelinepark/anaconda3/envs/scanpytest/lib/python3.6/site-packages/sklearn/ensemble/forest.py:465: RuntimeWarning: invalid value encountered in true_divide
  predictions[k].sum(axis=1)[:, np.newaxis])
/Users/madelinepark/anaconda3/envs/scanpytest/lib/python3.6/site-packages/sklearn/ensemble/forest.py:460: UserWarning: Some inputs do not have OOB scores. This probably means too few trees were used to compute any reliable oob estimates.
  warn("Some inputs do not have OOB scores. "
/Users/madelinepark/anaconda3/envs/scanpytest/lib/python3.6/site-packages/sklearn/ensemble/forest.py:465: RuntimeWarning: invalid value encountered in true_divide
  predictions[k].sum(axis=1)[:, np.newaxis])
/Users/madelinep

result writing
Index(['0610007N19Rik', '0610007P08Rik', '0610007P14Rik', '0610007P22Rik',
       '0610008F07Rik', '0610009B14Rik', '0610009B22Rik', '0610009D07Rik',
       '0610009L18Rik', '0610009O20Rik',
       ...
       'Zrsr1', 'Zrsr2', 'Zscan10', 'Zscan12', 'Zscan18', 'Zscan2', 'Zscan20',
       'Zscan21', 'Zscan29', 'zsGreen_transgene'],
      dtype='object', name='index', length=9162)
[0. 0. 0. ... 0. 0. 0.]
epithelial cell of proximal tubule
training...


/Users/madelinepark/anaconda3/envs/scanpytest/lib/python3.6/site-packages/sklearn/ensemble/forest.py:460: UserWarning: Some inputs do not have OOB scores. This probably means too few trees were used to compute any reliable oob estimates.
  warn("Some inputs do not have OOB scores. "
/Users/madelinepark/anaconda3/envs/scanpytest/lib/python3.6/site-packages/sklearn/ensemble/forest.py:465: RuntimeWarning: invalid value encountered in true_divide
  predictions[k].sum(axis=1)[:, np.newaxis])
/Users/madelinepark/anaconda3/envs/scanpytest/lib/python3.6/site-packages/sklearn/ensemble/forest.py:460: UserWarning: Some inputs do not have OOB scores. This probably means too few trees were used to compute any reliable oob estimates.
  warn("Some inputs do not have OOB scores. "
/Users/madelinepark/anaconda3/envs/scanpytest/lib/python3.6/site-packages/sklearn/ensemble/forest.py:465: RuntimeWarning: invalid value encountered in true_divide
  predictions[k].sum(axis=1)[:, np.newaxis])
/Users/madelinep

result writing
Index(['0610007N19Rik', '0610007P08Rik', '0610007P14Rik', '0610007P22Rik',
       '0610008F07Rik', '0610009B14Rik', '0610009B22Rik', '0610009D07Rik',
       '0610009L18Rik', '0610009O20Rik',
       ...
       'Zrsr1', 'Zrsr2', 'Zscan10', 'Zscan12', 'Zscan18', 'Zscan2', 'Zscan20',
       'Zscan21', 'Zscan29', 'zsGreen_transgene'],
      dtype='object', name='index', length=9162)
[0. 0. 0. ... 0. 0. 0.]


,Kidney-new-24m-cluster,Kidney-new-24m-cluster_gini,kidney loop of Henle ascending limb epithelial cell,kidney loop of Henle ascending limb epithelial cell_gini,fenestrated cell,fenestrated cell_gini,mesangial cell,mesangial cell_gini,B cell,B cell_gini,kidney collecting duct epithelial cell,kidney collecting duct epithelial cell_gini,T cell,T cell_gini,kidney collecting duct principal cell,kidney collecting duct principal cell_gini,macrophage,macrophage_gini,epithelial cell of proximal tubule,epithelial cell of proximal tubule_gini
0,0610007N19Rik,0.0,0610007N19Rik,0.0,0610007N19Rik,0.0,0610007N19Rik,0.0,0610007N19Rik,0.0,0610007N19Rik,0.0,0610007N19Rik,0.0,0610007N19Rik,0.0,0610007N19Rik,0.0,0610007N19Rik,0.0
1,0610007P08Rik,0.0,0610007P08Rik,0.0,0610007P08Rik,0.0,0610007P08Rik,0.0,0610007P08Rik,0.0,0610007P08Rik,0.0,0610007P08Rik,0.0,0610007P08Rik,0.0,0610007P08Rik,0.0,0610007P08Rik,0.0
2,0610007P14Rik,0.0,0610007P14Rik,0.0,0610007P14Rik,0.0,0610007P14Rik,0.0,0610007P14Rik,0.0,0610007P14Rik,0.0,0610007P14Rik,0.0,0610007P14Rik,0.0,0610007P14Rik,0.0,0610007P14Rik,0.0
3,0610007P22Rik,0.0,0610007P22Rik,0.0,0610007P22Rik,0.0,0610007P22Rik,0.0,0610007P22Rik,0.0,0610007P22Rik,0.0,0610007P22Rik,0.0,0610007P22Rik,0.0,0610007P22Rik,0.0,0610007P22Rik,0.0
4,0610008F07Rik,0.0,0610008F07Rik,0.0,0610008F07Rik,0.0,0610008F07Rik,0.0,0610008F07Rik,0.0,0610008F07Rik,0.0,0610008F07Rik,0.0,0610008F07Rik,0.0,0610008F07Rik,0.0,0610008F07Rik,0.0
5,0610009B14Rik,0.0,0610009B14Rik,0.0,0610009B14Rik,0.0,0610009B14Rik,0.0,0610009B14Rik,0.0,0610009B14Rik,0.0,0610009B14Rik,0.0,0610009B14Rik,0.0,0610009B14Rik,0.0,0610009B14Rik,0.0
6,0610009B22Rik,0.0,0610009B22Rik,0.0,0610009B22Rik,0.0,0610009B22Rik,0.0,0610009B22Rik,0.0,0610009B22Rik,0.0,0610009B22Rik,0.0,0610009B22Rik,0.0,0610009B22Rik,0.0,0610009B22Rik,0.0
7,0610009D07Rik,0.0,0610009D07Rik,0.0,0610009D07Rik,0.0,0610009D07Rik,0.0,0610009D07Rik,0.0,0610009D07Rik,0.0,0610009D07Rik,0.0,0610009D07Rik,0.0,0610009D07Rik,0.0,0610009D07Rik,0.0
8,0610009L18Rik,0.0,0610009L18Rik,0.0,0610009L18Rik,0.0,0610009L18Rik,0.0,0610009L18Rik,0.0,0610009L18Rik,0.0,0610009L18Rik,0.0,0610009L18Rik,0.0,0610009L18Rik,0.0,0610009L18Rik,0.0
9,0610009O20Rik,0.0,0610009O20Rik,0.0,0610009O20Rik,0.0,0610009O20Rik,0.0,0610009O20Rik,0.0,0610009O20Rik,0.0,0610009O20Rik,0.0,0610009O20Rik,0.0,0610009O20Rik,0.0,0610009O20Rik,0.0


In [35]:
results_cell_cv.to_csv('test3.csv')

In [36]:
tiss.obs

,FACS.selection,age,batch,cell,cell_ontology_class,cell_ontology_id,cellid,free_annotation,method,mouse.id,...,sex,subtissue,tissue,well,n_genes,n_counts,louvain,cluster_names,age_type_of_interest,cell_type_of_interest
index,,,,,,,,,,,,,,,,,,,,,
A11_B001261_S287_L003.mus-2-0,Viable,24m,0,nan,T cell,CL:0000084,A11_B001261,T cell,facs,24_59_M,...,male,NA,Kidney,A11,1488,1108365.0,8,8_T cell,rest,rest
A12_B001261_S288_L003.mus-2-0,Viable,24m,0,nan,epithelial cell of proximal tubule,CL:0002306,A12_B001261,Proximal tube,facs,24_59_M,...,male,NA,Kidney,A12,1855,2755523.0,5,5_epithelial cell of proximal tubule,rest,rest
A15_B001261_S291_L003.mus-2-0,Viable,24m,0,nan,kidney collecting duct principal cell,CL:1001431,A15_B001261,ductal princeple,facs,24_59_M,...,male,NA,Kidney,A15,718,708214.0,7,7_kidney collecting duct principal cell,rest,rest
A16_B001261_S292_L003.mus-2-0,Viable,24m,0,nan,T cell,CL:0000084,A16_B001261,T cell,facs,24_59_M,...,male,NA,Kidney,A16,2113,2041432.0,8,8_T cell,rest,rest
A17_B001261_S293_L003.mus-2-0,Viable,24m,0,nan,epithelial cell of proximal tubule,CL:0002306,A17_B001261,Proximal tube,facs,24_59_M,...,male,NA,Kidney,A17,1926,1743637.0,5,5_epithelial cell of proximal tubule,rest,rest
A18_B001261_S294_L003.mus-2-0,Viable,24m,0,nan,macrophage,CL:0000235,A18_B001261,macrophage,facs,24_59_M,...,male,NA,Kidney,A18,1722,1671482.0,6,6_macrophage,rest,rest
A20_B001261_S296_L003.mus-2-0,Viable,24m,0,nan,epithelial cell of proximal tubule,CL:0002306,A20_B001261,Proximal tube,facs,24_59_M,...,male,NA,Kidney,A20,1327,1335461.0,5,5_epithelial cell of proximal tubule,rest,rest
A21_B001261_S297_L003.mus-2-0,Viable,24m,0,nan,T cell,CL:0000084,A21_B001261,T cell,facs,24_59_M,...,male,NA,Kidney,A21,1991,1374989.0,8,8_T cell,rest,rest
A3_B001261_S279_L003.mus-2-0,Viable,24m,0,nan,epithelial cell of proximal tubule,CL:0002306,A3_B001261,Proximal tube,facs,24_59_M,...,male,NA,Kidney,A3,1358,953170.0,2,2_epithelial cell of proximal tubule,rest,rest
